# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [9]:
import folium

# Create a map centered at an arbitrary location (e.g., the first city's coordinates)
m = folium.Map(location=[city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]], zoom_start=4)

# Add markers for each city with humidity-based size
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 5,  # Adjust the scaling for appropriate marker size
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=row['City']
    ).add_to(m)

# Display the map
m


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Humidity'] <= 80) &
    (city_data_df['Max Temp'] >= 10) &  
    (city_data_df['Max Temp'] <= 60)
].copy()  # Create a copy of the filtered DataFrame

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
2,new norfolk,AU,-42.7826,147.0587,58,
4,lanzhou,CN,36.0564,103.7922,48,
6,albany,US,42.6001,-73.9662,74,
7,carnarvon,AU,-24.8667,113.6333,76,
...,...,...,...,...,...,...
566,nhulunbuy,AU,-12.2333,136.7667,78,
568,road town,VG,18.4167,-64.6167,70,
574,canchungo,GW,12.0672,-16.0333,61,
575,iquique,CL,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 100000  # 100,000 meters
params = {
    "type": "hotel",
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting Hotel Search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distanceToPoint({lng},{lat}) lt {radius}"
    params["bias"] = f"proximity({lng},{lat})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(f"{base_url}/nearby", params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["features"][0]["properties"]["formatted"]
        hotel_df.loc[index, "Hotel Lat"] = name_address["features"][0]["geometry"]["coordinates"][1]
        hotel_df.loc[index, "Hotel Lng"] = name_address["features"][0]["geometry"]["coordinates"][0]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No Hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
#This code iterates through each city's coordinates in the hotel_df DataFrame, sends an API request to Geoapify's Places API to find nearby hotels, and updates the DataFrame with hotel information. It logs the search results and displays the updated hotel_df DataFrame.

#Make sure to replace 'your_geoapify_api_key' with your actual Geoapify API key and adjust any other parts of the code as needed based on the API's response structure or your specific requirements.







Starting Hotel Search
faya - nearest hotel: No Hotel found
new norfolk - nearest hotel: No Hotel found
lanzhou - nearest hotel: No Hotel found
albany - nearest hotel: No Hotel found
carnarvon - nearest hotel: No Hotel found
saint-pierre - nearest hotel: No Hotel found
trairi - nearest hotel: No Hotel found
avarua - nearest hotel: No Hotel found
guiratinga - nearest hotel: No Hotel found
rikitea - nearest hotel: No Hotel found
constitucion - nearest hotel: No Hotel found
bambous virieux - nearest hotel: No Hotel found
zangakatun - nearest hotel: No Hotel found
saint-louis - nearest hotel: No Hotel found
grimmen - nearest hotel: No Hotel found
cape town - nearest hotel: No Hotel found
stornoway - nearest hotel: No Hotel found
naberera - nearest hotel: No Hotel found
drochia - nearest hotel: No Hotel found
tomatlan - nearest hotel: No Hotel found
sao joao da barra - nearest hotel: No Hotel found
margate - nearest hotel: No Hotel found
ancud - nearest hotel: No Hotel found
dalby - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No Hotel found
2,new norfolk,AU,-42.7826,147.0587,58,No Hotel found
4,lanzhou,CN,36.0564,103.7922,48,No Hotel found
6,albany,US,42.6001,-73.9662,74,No Hotel found
7,carnarvon,AU,-24.8667,113.6333,76,No Hotel found
...,...,...,...,...,...,...
566,nhulunbuy,AU,-12.2333,136.7667,78,No Hotel found
568,road town,VG,18.4167,-64.6167,70,No Hotel found
574,canchungo,GW,12.0672,-16.0333,61,No Hotel found
575,iquique,CL,-20.2208,-70.1431,68,No Hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
import folium

# Create a map centered at an arbitrary location (e.g., the first city's coordinates)
m = folium.Map(location=[city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]], zoom_start=4)

# Iterate through the city_data_df DataFrame
for index, city_row in city_data_df.iterrows():
    city_name = city_row['City']
    
    # Find the hotel row that matches the city name
    hotel_row = hotel_df[hotel_df['City'] == city_name]
    
    if not hotel_row.empty:
        hotel_info = hotel_row.iloc[0]
        
        folium.CircleMarker(
            location=[city_row['Lat'], city_row['Lng']],
            radius=city_row['Humidity'] / 5,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            popup=f"City: {city_name}<br>Country: {city_row['Country']}<br>Humidity: {city_row['Humidity']}<br>Hotel Name: {hotel_info['Hotel Name']}"
        ).add_to(m)
    else:
        folium.CircleMarker(
            location=[city_row['Lat'], city_row['Lng']],
            radius=city_row['Humidity'] / 5,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            popup=f"City: {city_name}<br>Country: {city_row['Country']}<br>Humidity: {city_row['Humidity']}<br>Hotel Name: No hotel found"
        ).add_to(m)

# Save the map to an HTML file
m.save("../output_data/map_with_hotels.html")

